In [44]:
import chromadb
import ollama
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import pandas as pd

In [45]:
def read_vectordb_as_df(db_path:str):
    client = chromadb.PersistentClient(path=db_path)
    for collection in client.list_collections():
        data = collection.get(include=['embeddings', 'documents', 'metadatas'])
        df = pd.DataFrame({"ids":data["ids"], 
                            "metadatas":data["metadatas"], 
                            "documents":data["documents"]})
        df["first_div"] = df["metadatas"].apply(lambda x: x["First Division"])
        df["second_div"] = df["metadatas"].apply(lambda x: x["Second Division"])
        df["filename"] = df["metadatas"].apply(lambda x: x["File Name"])
        df = df[["ids", "first_div", "second_div","filename","documents", "metadatas"]]
    return df

In [46]:
db_path = "./db/chroma_db_02"
vector_store = Chroma(collection_name="my_collection", persist_directory=db_path, embedding_function=OllamaEmbeddings(model="bge-m3:latest"))
print(vector_store)

In [47]:
df = read_vectordb_as_df(db_path=db_path)
df.head(2)

,ids,first_div,second_div,filename,documents,metadatas
0,faace8c4-ab2c-43b4-9b4e-7fc15319bc78,MANUAL,Common,[KISA Insight 2023 Vol.03] ChatGPT(챗GPT) 보ᄋ...,This page explains [KISA Insight 2023 Vol.03] ...,{'File Name': '[KISA Insight 2023 Vol.03] Chat...
1,1a3d1b93-e5d3-4a96-990d-e4ba6b976e29,MANUAL,Common,[KISA Insight 2023 Vol.03] ChatGPT(챗GPT) 보ᄋ...,This page explains [KISA Insight 2023 Vol.03] ...,{'File Name': '[KISA Insight 2023 Vol.03] Chat...


In [9]:
# d = "I like a apple"
# response = ollama.embeddings(model="bge-m3:latest", prompt=d)
# response

In [48]:
import numpy as np
from sentence_transformers.util import cos_sim

def get_similarity_search_score_rank(query:str, db_path:str):
    embedded_query = ollama.embeddings(model="bge-m3:latest", prompt=query)
    embedded_query=[np.float64(k) for k in embedded_query['embedding']]

    client = chromadb.PersistentClient(path=db_path)
    for collection in client.list_collections():
        data = collection.get(include=['embeddings'])
    
    results = []
    for d in data['embeddings']:
        similarity = cos_sim(embedded_query, d)
        results.append(similarity)

    results = [r.item() for r in results]

    sorted_indices = np.argsort(results)  # 값에 대한 정렬된 인덱스
    order_values = np.empty_like(sorted_indices)
    order_values[sorted_indices] = np.arange(len(results))

    return order_values


query = """
what is the noon report in iss system?
"""
res1 = get_similarity_search_score_rank(query=query, db_path=db_path)
res1

array([47166, 44995, 53152, ..., 30774, 28023,  5082], dtype=int64)

In [49]:
len(res1)

53790

In [56]:
import re
import numpy as np
from rank_bm25 import BM25Okapi

def bm25_search_rank(query:str, db_path:str):
    client = chromadb.PersistentClient(path=db_path)
    for collection in client.list_collections():
        data = collection.get(include=['documents', 'metadatas'])
    tokenized_corpus = [doc.split(" ") for doc in data["documents"]]
    bm25 = BM25Okapi(tokenized_corpus)

    pattern = r'"(.*?)"'  # 따옴표로 둘러싸인 단어만 검색 대상으로 리스트에 담기
    tokenized_query = re.findall(pattern, query)
    print(tokenized_query)

    doc_scores = bm25.get_scores(tokenized_query)

    sorted_indices = np.argsort(doc_scores)  # 값에 대한 정렬된 인덱스
    order_values = np.empty_like(sorted_indices)
    order_values[sorted_indices] = np.arange(len(doc_scores))


    return order_values

res2 = bm25_search_rank(query='"what" is the "noon" "report" in "iss" system', db_path="./db/chroma_db_02")
res2

['what', 'noon', 'report', 'iss']


array([    0, 35667, 35668, ..., 17868,  8965, 52566], dtype=int64)

In [57]:
len(res2)

53790

In [58]:
def rrf(all_rankings: list[list[int]]):
    """Takes in list of rankings produced by multiple retrieval algorithms,
    and returns newly of ranked and scored items."""
    scores = {} # key is the index and value is the score of that index
    # 1. Take every retrieval algorithm ranking
    for algorithm_ranks in all_rankings:
        # 2. For each ranking, take the index and the ranked position
        for rank, idx in enumerate(algorithm_ranks):
            # 3. Calculate the score and add it to the index
            if idx in scores:
                scores[idx] += 1 / (60 + rank)
            else:
                scores[idx] = 1 / (60 + rank)

    # 4. Sort the indices based on accumulated scores
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    return sorted_scores

In [59]:
new_ranks = rrf([res1, res2])
new_ranks[:10]

[(35429, 0.01699409210619973),
 (47166, 0.016729158855143107),
 (0, 0.016721766121182067),
 (44995, 0.016431307068236697),
 (35667, 0.0164252958433114),
 (53152, 0.016183638275647654),
 (35668, 0.01615240548169952),
 (35669, 0.015965420234501735),
 (35670, 0.015666296716911006),
 (193, 0.01565292672028597)]

In [60]:
df["metadatas"][193]

{'File Name': 'Integrated Smart Ship(ISS1.0)',
 'File Path': '/content/drive/MyDrive/MANUAL/Integrated Smart Ship(ISS)/Integrated Smart Ship(ISS1.0).pdf',
 'First Division': 'MANUAL',
 'Page': 89,
 'Second Division': 'Integrated Smart Ship(ISS)'}

In [42]:
df["metadatas"][47166]

{'File Name': 'RULES FOR THE CLASSIFICATION OF STEEL SHIPS_467-NR_PartC_2024-07',
 'File Path': '/content/drive/MyDrive/Rules/BV/RULES FOR THE CLASSIFICATION OF STEEL SHIPS_467-NR_PartC_2024-07.pdf',
 'First Division': 'Rules',
 'Page': 48,
 'Second Division': 'BV'}

In [61]:
df["metadatas"][0]

{'File Name': '[KISA Insight 2023 Vol.03] ChatGPT(챗GPT) 보안 위협과 시사점',
 'File Path': '/content/drive/MyDrive/MANUAL/Common/[KISA Insight 2023 Vol.03] ChatGPT(챗GPT) 보안 위협과 시사점.pdf',
 'First Division': 'MANUAL',
 'Page': 0,
 'Second Division': 'Common'}